# Travel Product Ideas

In this notebook, we run market research simulations for which we have (or believe to have :-) the correct answers.

In [1]:
from tinytroupe.agent import TinyPerson
from tinytroupe.environment import TinyWorld
from tinytroupe import control
from tinytroupe import config_manager


from tinytroupe.experimentation import InPlaceExperimentRunner

# specific utilities for the usecase
from market_research_utils import *


!!!!
DISCLAIMER: TinyTroupe relies on Artificial Intelligence (AI) models to generate content. 
The AI models are not perfect and may produce inappropriate or inacurate results. 
For any serious or consequential use, please review the generated content before using it.
!!!!

Looking for default config on: C:\Users\pdasilva\repos\TinyTroupe\tinytroupe\utils\..\config.ini
Found custom config on: c:\Users\pdasilva\OneDrive - Microsoft\TinyTroupe (shared)\Paper artifacts\Working examples (GPT-5)\config.ini
TinyTroupe version: 0.6.0
Current date and time (local): 2025-10-04 10:53:07
Current date and time (UTC):   2025-10-04 13:53:07

Current TinyTroupe configuration 
[OpenAI]
api_type = azure
azure_api_version = 2024-12-01-preview
model = gpt-5-mini
reasoning_model = o3-mini
embedding_model = text-embedding-3-small
max_completion_tokens = 128000
timeout = 300
max_attempts = 5
waiting_time = 0
exponential_backoff_factor = 5
reasoning_effort = high
cache_api_calls = False
cache_file_name = o

In [2]:
config_manager.update("timeout", 300)
config_manager.update("api_type", "azure")
config_manager.update("waiting_time", 5)

2025-10-04 10:53:30,334 - root - INFO - Updated config: timeout = 300
2025-10-04 10:53:30,335 - root - INFO - Updated config: api_type = azure
2025-10-04 10:53:30,336 - root - INFO - Updated config: waiting_time = 5


In [3]:
#control.begin("travel_product_market_research_2bb.cache.json")

## Experiment setup

In [4]:
#Experiment: Travel Preferences by Marital Status
#	•	Ground Truth:
#	•	Singles prefer adventure trips, with 62% citing nature or extreme sports as their ideal vacation (Expedia, 2023). Source
#	•	Families prioritize child-friendly destinations, with 73% choosing theme parks or resorts (Expedia, 2023). Source
#	•	Couples favor luxury/romantic getaways, with 81% preferring beachfront or spa destinations (Expedia, 2023). Source
#	•	Prediction Task: Predict travel preferences based on marital status, income, and age.
#	•	Hypothetical Product:
# “WanderLux”: A travel service that helps setup luxury vacations and getaways, focusing on beachfront and spa destinations.


# this is meant to prime the agent to start thinking about what he/she would like to do on vacation
interviewer_introduction =\
    """
    We are performing some market research and need to know you more. 
    Can you please:
      - present yourself?
      - list your top-10 interests?
      - tell us what an ideal vacation would be for you and why?
    """

interviewer_main_question =\
    """ 
    We are developing a new travel service called "WanderLux" that helps setup luxury or romantic vacations, 
    focusing on beachfront and spa destinations.  The idea is to help people seeking quiet and calm getaways to 
    find the best places to stay and relax, avoiding fake reviews and bad experiences. These places are meant for 
    an adult audience, many not accepting children.
    
    The question is: if you had only one vacation to take next year, would use our new service
    or not? You **must** respond with "Yes" or "No" **only**, and explain why. If you are not sure, 
    please pick the answer that you think is most likely. 

    Please be honest, we are not here to judge you, but just to learn from you. We know your preferences and choices depend on many factors, 
    but please make your best guess, and you **must** think deeply about the following before answering:

       - your current situation in life, location, job and interests.
       = what you look for in vacations. Are you sure quiet luxury is what you want, instead of, say, adventure or fun and parties?
       - what would be OTHER vacation alternatives you might prefer instead.
       - and what you can afford.
     
    Now please answer the question.
    """

Now let's enclose the audience in an environment, to make it easier to interact with. Note that we set `broadcast_if_no_target` to `False`, which instructs the environment to **do not** broadcast what specific agents say to the others (unless the agent deliberately specify the target). We are not simulating a focus group, so we don't want discussions.


In [5]:
import random

usa_singles = random.sample(TinyPerson.load_specifications_from_folder("./population/usa_singles_2/"), 50)  # no children
usa_couples = random.sample(TinyPerson.load_specifications_from_folder("./population/usa_couples_2/"), 50)   # no children
usa_families = random.sample(TinyPerson.load_specifications_from_folder("./population/usa_families_2b/"), 50) # have children, emphasize family and childrearing aspects

In [6]:
# ENABLE ERROR CORRECTION
for population in [usa_singles, usa_couples, usa_families]:
    for person in population:
       person.action_generator.enable_reasoning_step = False
       person.action_generator.enable_quality_checks = True
       person.action_generator.max_attempts = 5
       person.action_generator.enable_regeneration = True
       person.action_generator.quality_threshold = 7 #5

In [7]:
singles_market = TinyWorld(f"Target audience 1 (US)", usa_singles, broadcast_if_no_target=False)
families_market = TinyWorld(f"Target audience 2 (US)", usa_families, broadcast_if_no_target=False)
couples_market = TinyWorld(f"Target audience 3 (US)", usa_couples, broadcast_if_no_target=False)

## Perform the research

Now we actually ask agents about their opinion. Note that we are asking for specific answers (Yes, No, Maybe), both to convey the granularity of preference we seek, and to make it easier to extract the results later. In general, a coarser granuarity is likely to have higher quality, since it requires less information to be computed accurately.

In [8]:
singles_market.broadcast(interviewer_introduction)
families_market.broadcast(interviewer_introduction)
couples_market.broadcast(interviewer_introduction)

singles_market.broadcast(interviewer_main_question)
families_market.broadcast(interviewer_main_question)
couples_market.broadcast(interviewer_main_question)

USER --> Lillian Porter: [CONVERSATION] 
          >      We are performing some market research and need to know you more.      Can you
          > please:       - present yourself?       - list your top-10 interests?       - tell us
          > what an ideal vacation would be for you and why?

       + --> Noelani Benally

       + --> Lucas Ramirez

       + --> ...others...

USER --> Earl Benjamin Crockett: [CONVERSATION] 
          >      We are performing some market research and need to know you more.      Can you
          > please:       - present yourself?       - list your top-10 interests?       - tell us
          > what an ideal vacation would be for you and why?

...

       + --> ...others...

USER --> Owen Larson: [CONVERSATION] 
          >       We are developing a new travel service called "WanderLux" that helps setup luxury
          > or romantic vacations,      focusing on beachfront and spa destinations.  The idea is to
          > help people seeking quiet and calm getaways to      find the best places to stay and
          > relax, avoiding fake reviews and bad experiences. These places are meant for      an
          > adult audience, many not accepting children.          The question is: if you had only
          > one vacation to take next year, would use our new service     or not? You **must**
          > respond with "Yes" or "No" **only**, and explain why. If you are not sure,      please
          > pick the answer that you think is most likely.       Please be honest, we are not here
          > to judge you, but just to learn from you. We know your preferences and choices depend on
          > many factors,      but please make your best guess, and you **must** think deeply about
          > the following before answering:         - your current situation in life, location, job
          > and interests.        = what you look for in vacations. Are you sure (...)

       + --> Isabella Carter

       + --> Liam Patel-Smith

       + --> ...others...

In [ ]:
families_market.run(1)
#control.checkpoint()

2025-10-04 10:53:38,649 - tinytroupe - INFO - [Target audience 2 (US)] Running world simulation step 1 of 1.


─────────────────────────────────────── Target audience 2 (US) step 1 of 1 ────────────────────────────────────────

2025-10-04 10:53:38,654 - tinytroupe - INFO - [Target audience 2 (US)] No timedelta provided, so the datetime was not advanced.
2025-10-04 10:53:38,669 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,679 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,692 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,704 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,712 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,716 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,721 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,727 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,731 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 10:53:38,735 - tinytroupe - INFO - Using Azure OpenAI Servic

Maritza Elena Rios acts: [THINK] 
                        > I need to answer honestly. Consider my life: I work part-time as a
                        > preschool assistant in South LA, have two small children (Sofía 4.5 and
                        > Mateo 2), and our household budget is tight (~$28k/year). WanderLux is
                        > adult-only, luxury beachfront/spa stays that often don't accept children.
                        > Practical points: 1) Cost — luxury trips are likely out of our budget
                        > unless there's a big gift or savings; 2) Childcare — reliable, affordable
                        > childcare for several days is rare and stressful; I would feel guilty
                        > leaving the kids unless family could help; 3) Preferences — I usually
                        > prefer short road trips, family-friendly places with a kitchenette, or a
                        > one-night local getaway with family help; 4) Alternatives — visit
                        > grandparents, a short drive to a nearby beach or splash pad, a local hotel
                        > staycation with on-site laundry and kitchenette, or a family-friendly
                        > rental; 5) Could I be convinced? If the service arranged affordable
                        > childcare or offered short, nearby low-cost options, maybe, but as
                        > described it's not a good fit. Conclu (...)

Maritza Elena Rios acts: [TALK] 
                        > No — I would not use WanderLux. Between childcare limits, the cost of
                        > luxury beachfront/spa trips, and how much I value being with my children,
                        > it isn't a realistic fit for my family. I prefer short, low-stress trips
                        > that work with nap schedules (a 1–3 hour drive to a nearby beach, staying
                        > with family, or a local hotel with a kitchenette and laundry). If the
                        > service offered more affordable, family-friendly or very short nearby
                        > adults-getaway options plus help with reliable childcare, I might
                        > reconsider, but as described, no.

...

2025-10-04 11:24:32,326 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 11:24:32,669 - tinytroupe - INFO - Waiting 5 seconds before next API request (to avoid throttling)...
2025-10-04 11:24:41,121 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Mark Allen Hargrove acts: [THINK] 
                         > Okay — weigh this up like I would at the kitchen table. Family first: two
                         > little kids (4 and 2), Sarah handles most scheduling, and I can't
                         > realistically do a week-long adults-only beachfront trip without
                         > childcare help or Grandma/Grandpa stepping in. I'm budget-conscious —
                         > combined income around $68k, saving for mortgage and college fund. I like
                         > quiet and calm sometimes (early-morning fishing, Hocking Hills cabin),
                         > but when we take a big vacation I usually want it to be family-friendly
                         > so the kids get something out of it too. WanderLux sounds like an adult-
                         > only luxury/spa service that often won't accept kids; that rules it out
                         > for a family trip unless it were a short couple's getaway and we could
                         > afford babysitting and the price. Other alternatives I'd pick: a nearby
                         > cabin (Hocking Hills), Put-in-Bay weekend, or visiting grandparents — all
                         > cheaper, closer, kid-friendly. So my best, honest pick: No. I'll explain
                         > that succinctly.

Mark Allen Hargrove acts: [TALK] 
                         > No — I wouldn’t use WanderLux for my one vacation next year. With two
                         > little kids and a tight budget we’d more likely pick a family-friendly,
                         > driveable trip (a cabin in Hocking Hills, Put‑in‑Bay, or visiting
                         > grandparents) so the kids can come and we don’t burn a big chunk of
                         > savings on childcare and luxury rates. If Sarah and I could afford a
                         > short, paid-for couples’ getaway and had reliable babysitting lined up, I
                         > might consider a service like yours for a one-off romantic trip, but
                         > given our current priorities and costs, a family-focused, nearby option
                         > is more likely.

Mark Allen Hargrove acts: [DONE] 
                         > Finished answering the market-research question.

2025-10-04 11:24:41,172 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 11:24:41,579 - tinytroupe - INFO - Waiting 5 seconds before next API request (to avoid throttling)...
2025-10-04 11:24:52,603 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-10-04 11:24:52,651 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 11:24:53,068 - tinytroupe - INFO - Waiting 5 seconds before next API request (to avoid throttling)...


In [ ]:
couples_market.run(1)
#control.checkpoint()

2025-10-04 09:33:22,274 - tinytroupe - INFO - [Target audience 3 (US)] Running world simulation step 1 of 1.


─────────────────────────────────────── Target audience 3 (US) step 1 of 1 ────────────────────────────────────────

2025-10-04 09:33:22,279 - tinytroupe - INFO - [Target audience 3 (US)] No timedelta provided, so the datetime was not advanced.
2025-10-04 09:33:22,296 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,311 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,333 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,341 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,368 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,408 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,484 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,489 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,490 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:33:22,500 - tinytroupe - INFO - Using Azure OpenAI Servic

Yvonne Harris acts: [THINK] 
                   > I need to decide if Yvonne would use a luxury beachfront/spa service next year.
                   > Consider her age (72), retirement, modest fixed income, and long-term travel
                   > goal: one more trip with Harold to visit relatives in the South. She likes
                   > quiet and calm but prefers meaningful, community-centered trips (family,
                   > church, local markets) over flashy luxury. Cost matters — she is financially
                   > prudent and wary of large, costly vacations. She also trusts personal
                   > recommendations (church, neighbors) more than unfamiliar services, though she
                   > would appreciate help avoiding scams. Other vacation options she prefers: a
                   > road trip to see family, modest B&B stays, a genealogy trip, or a short
                   > inexpensive getaway to a nearby town or farmer's market. Conclusion: most
                   > likely No, unless the service offered a very affordable, family-visit-friendly
                   > package or came highly recommended by trusted community contacts.

Yvonne Harris acts: [TALK] 
                   > No. I wouldn’t likely use WanderLux for a vacation next year, honey. Harold and
                   > I are more apt to take a trip that reconnects us with family down South or a
                   > simple, modest getaway — places that feel like home and don’t cost the moon.
                   > I’m careful with our retirement savings and tend to trust recommendations from
                   > church and neighbors over unfamiliar luxury services. Now, if WanderLux had an
                   > affordable package that helped us visit relatives safely and without fuss, and
                   > came recommended by people I trust, I might consider it. But as a beachfront,
                   > adults-only luxury/spa service aimed at romantic getaways, it’s not what I’m
                   > most likely to pick.

...

Nina Johnson-Rivera acts: [DONE] 
                         > Waiting for follow-up questions.

2025-10-04 09:39:56,879 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-10-04 09:39:56,894 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:39:57,258 - tinytroupe - INFO - Waiting 5 seconds before next API request (to avoid throttling)...
2025-10-04 09:40:14,982 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-10-04 09:40:15,006 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:40:15,301 - tinytroupe - INFO - Waiting 5 seconds before next API request (to avoid throttling)...
2025-10-04 09:40:24,319 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "H

Levi Nelson acts: [THINK] 
                 > Okay — I'm Levi, 14, living in a tight-budget household. Think it through: 1) Who
                 > WanderLux targets: adult, luxury, beachfront/spa, often no kids allowed — not
                 > aimed at teens. 2) My priorities: save for a truck, help at home, like
                 > fishing/creaky garages and county fairs — I want hands-on or outdoors trips, not
                 > quiet spa luxury. 3) Money: family income is low, we can't afford fancy
                 > vacations; I put odd-job money in a 'Truck Fund'. 4) Other options I’d prefer: a
                 > cheap road trip to a lake for fishing, camping, visiting grandparents, or the
                 > county fair — stuff I can do with friends or family and not cost a ton. 5) Edge
                 > cases: I might use a service like that only if it was a gift or my parents paid
                 > and it fit our needs, but that's unlikely. Conclusion: answer is No.

Levi Nelson acts: [TALK] 
                 > No. I'm 14 and not the kind of customer you want — WanderLux sounds like an
                 > adult-focused, pricey spa/beach service, which I can't afford and wouldn't pick
                 > anyway. I prefer simple, active trips (fishing, camping, visiting my
                 > grandparents, or the county fair) and I'm saving my odd-job money for a pickup.
                 > If my family could afford a fancy trip as a gift someday, maybe it'd be nice, but
                 > right now it's not for me.

Levi Nelson acts: [DONE] 
                 > Finished answering the WanderLux survey honestly.

In [ ]:
singles_market.run(1)
#control.checkpoint()

2025-10-04 09:01:57,910 - tinytroupe - INFO - [Target audience 1 (US)] Running world simulation step 1 of 1.


─────────────────────────────────────── Target audience 1 (US) step 1 of 1 ────────────────────────────────────────

2025-10-04 09:01:57,915 - tinytroupe - INFO - [Target audience 1 (US)] No timedelta provided, so the datetime was not advanced.
2025-10-04 09:01:57,927 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,935 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,951 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,965 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,972 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,978 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,988 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,992 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:57,997 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:01:58,000 - tinytroupe - INFO - Using Azure OpenAI Servic

Isaac Johnson acts: [THINK] 
                   > Step-by-step reasoning about WanderLux request: 1) My situation: I'm 30, living
                   > in Los Angeles, currently unemployed after leaving a part-time retail job due
                   > to burnout. I'm actively looking for work in graphic design and trying to save
                   > where I can. Budget is a real constraint. 2) Preferences: I like quiet,
                   > creative, restorative experiences — cozy cafes, nature walks, calm beaches, and
                   > spa vibes. A vetted, adult-only luxury beach/spa getaway fits what I find
                   > restorative and appealing. 3) Trade-offs: Luxury/romantic beachfront vacations
                   > are likely expensive. With only one vacation next year, I have to choose
                   > between splurging on a luxury service or choosing something more affordable
                   > that still restores me (road trip to a nearby beach, camping, a local spa day,
                   > or a staycation focused on art and rest). 4) Trust factor: I value services
                   > that avoid fake reviews and curate authentic places — that's a plus for
                   > WanderLux and aligns with my dislike of shallow or misleading listings. 5)
                   > Likelihood: Given my current un (...)

Isaac Johnson acts: [TALK] 
                   > No. I really like the concept — quiet beachfront spas with vetted reviews
                   > sounds perfect for recharging — but right now I'm scraping together savings
                   > while job-hunting in LA, so I wouldn't use an expensive luxury service for my
                   > one vacation next year. I'd probably choose a cheaper restorative option (a
                   > nearby road trip, camping by the coast, a local spa day, or a dedicated
                   > staycation focused on art and rest) that gives the calm I want without blowing
                   > my savings. If my financial situation improves or it was a special celebration,
                   > I'd consider WanderLux in the future.

...

2025-10-04 09:02:57,505 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:02:57,933 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 429 Too Many Requests"
2025-10-04 09:02:57,943 - tinytroupe - WARNING - [2] Rate limit error, waiting a bit and trying again.
2025-10-04 09:02:57,943 - tinytroupe - INFO - Request failed. Waiting 10.0 seconds between requests...
2025-10-04 09:02:58,493 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 429 Too Many Requests"
2025-10-04 09:02:58,496 - tinytroupe - WARNING - [1] Rate limit error, waiting a bit and trying again.
2025-10-04 09:02:58,497 - tinytroupe - INFO - Request failed. Waiting 2 seconds between requests...
2025-10-04 09:02:58,669 - tinytroupe - INFO - Waiting 50.0 secon

Isabella Wright acts: [THINK] 
                     > Step-by-step: 1) My situation — I work long days at a warehouse in Philly,
                     > physical job, limited disposable income. 2) What I look for — I like getting
                     > away to stay active (basketball, hiking, camping) and to hang with friends;
                     > relaxation is nice but I prefer being social and moving. 3) Affordability —
                     > full-on luxury beachfront and spa trips usually cost more than I'd be
                     > comfortable spending for a single vacation. 4) Alternatives — a weekend
                     > basketball trip with buddies, a hiking/camping getaway, or a modest beach
                     > trip with friends are more likely choices. 5) Fit with WanderLux — the
                     > service sounds great for folks who want quiet luxury and can afford it, and
                     > who value vetted, adult-only spots. Given my preferences and budget, the
                     > service doesn't match what I'm most likely to pick. Conclusion: Most likely
                     > answer is No.

Isabella Wright acts: [TALK] 
                     > No. I wouldn't use WanderLux for my one vacation next year — nice idea, but
                     > I'm a 45-year-old warehouse worker on a budget who prefers active, social
                     > getaways (playing ball with friends, hiking, a simple beach weekend). Luxury
                     > adult-only beachfront/spa spots sound relaxing, but they're likely too pricey
                     > and not the kind of trip that recharges me. I'd pick a more affordable,
                     > activity-focused trip or something I can share with my crew.

Isabella Wright acts: [DONE] 
                     > Waiting for any follow-up questions.

2025-10-04 09:03:17,330 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:03:24,152 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-10-04 09:03:24,170 - tinytroupe - INFO - Using Azure OpenAI Service API with key...
2025-10-04 09:03:27,152 - httpx - INFO - HTTP Request: POST https://azureai-prototyping-ai.openai.azure.com/openai/deployments/gpt-5-mini/chat/completions?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
2025-10-04 09:03:27,169 - tinytroupe - INFO - Using Azure OpenAI Service API with key...


## Extract results and analyze

Now we can actually extract the results.

In [ ]:
df_singles = extract_and_analyze_results(usa_singles, "Singles")
#control.checkpoint()


In [ ]:
df_families = extract_and_analyze_results(usa_families, "Families")
#control.checkpoint()

In [ ]:
df_couples = extract_and_analyze_results(usa_couples, "Couples")
#control.checkpoint()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib
import numpy as np

def plot_combined_responses(title, name_to_df, individual_responses=None):
    """
    Combine the given dataframes in a single chart with grouped bars.
    Each dataframe is a different color and is properly labeled.
    
    Parameters:
    -----------
    title : str
        Title for the plot
    name_to_df : dict
        Dictionary mapping names to dataframes
    individual_responses : list, optional
        List of responses to count individually. If None, only 'Yes' and 'No' are counted individually.
        All other responses will be grouped as "Others".
    """
    print(f"################# Plotting combined responses for {title}... #################")
    
    # If individual_responses is not provided, default to Yes and No
    if individual_responses is None:
        individual_responses = ["Yes", "No"]
    
    # Get unique categories (individual responses + "Others")
    categories = individual_responses.copy()
    
    # Prepare data for grouped bar chart
    fig, ax = plt.subplots(figsize=(10, 5))
    
    # Define a colormap
    cmap = matplotlib.colormaps['tab10']
    
    # Calculate bar width based on number of groups
    num_groups = len(name_to_df)
    bar_width = 0.8 / num_groups  # Adjust the 0.8 to control spacing
    
    # Check if we need to add "Others" category
    has_others = False
    for df in name_to_df.values():
        if any(resp not in [r.lower() for r in individual_responses] for resp in [r.lower() for r in df["response"].unique()]):
            has_others = True
            break
    
    if has_others and "Others" not in categories:
        categories.append("Others")
    
    # For each category, plot grouped bars for each dataframe
    for i, category in enumerate(categories):
        for j, (name, df) in enumerate(name_to_df.items()):
            # Calculate the position for this bar
            position = i + (j - num_groups/2 + 0.5) * bar_width
            
            # Count occurrences for this category
            if category == "Others":
                # Count all responses that are not in individual_responses (case-insensitive)
                count = sum(df["response"].value_counts().get(resp, 0) 
                           for resp in df["response"].unique() 
                           if not any(resp.lower() == ind.lower() for ind in individual_responses))
            else:
                # Count occurrences of this category (case-insensitive)
                count = sum(df["response"].value_counts().get(resp, 0) 
                           for resp in df["response"].unique() 
                           if resp.lower() == category.lower())
            
            # Plot the bar
            ax.bar(position, count, width=bar_width, alpha=1.0, 
                   label=name if i == 0 else "", color=cmap(j % 10))
    
    # Set the x-ticks in the middle of each group
    ax.set_xticks(np.arange(len(categories)))
    ax.set_xticklabels(categories, fontsize=12)
    
    # Increase font sizes for better readability
    plt.title(f"Combined Responses ({title})", fontsize=16)
    plt.xlabel("Response", fontsize=14)
    plt.ylabel("Count", fontsize=14)
    plt.legend(fontsize=12)
    
    # Also increase tick label sizes
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_combined_responses("travel preferences by group", 
                        {"Singles (no children)": df_singles, 
                         "Couples (no children)": df_couples,
                         "Families (with children)": df_families})

In [ ]:
# statistics about the DF
df_singles.value_counts("response")

In [ ]:
df_couples.value_counts("response")

In [ ]:
df_families.value_counts("response")

In [ ]:
#control.end()

## Compare with Empirical Data (if any)